# 점수 매기기 프로필 (Scoring Profile)

이 노트북에서는 Azure AI Search의 Scoring Profile을 생성하여 검색 품질을 향상시킵니다.

**Scoring Profile 가중치 설정:**
- `name`: 3배 (가장 중요)
- `brand`: 2배 (중요)
- `description`: 1배 (기본)

점수 매기기 프로필은 인덱스 레벨에서 정의되어 여러 검색에서 재사용할 수 있습니다.

## 0. 필요한 라이브러리 설치

In [1]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 설정

In [2]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

Search Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index: products-index
OpenAI Endpoint: https://open-ai-seoulcentral-1023.openai.azure.com/
Embedding Model: text-embedding-3-large


## 2. 클라이언트 초기화

In [3]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from openai import AzureOpenAI
from IPython.display import display, HTML
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    credential=credential
)

openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 2-2. 임베딩 생성 함수
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

print("클라이언트 초기화 완료")

클라이언트 초기화 완료


## 3. Scoring Profile 생성

필드별 가중치를 정의하는 Scoring Profile을 인덱스에 추가합니다.

In [4]:
from azure.search.documents.indexes.models import (
    ScoringProfile,
    TextWeights
)

##############################################
# 3. Scoring Profile 생성 및 추가
##############################################

# 기존 인덱스 가져오기
existing_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

# Scoring Profile 정의
scoring_profile = ScoringProfile(
    name="product-weighted-profile",
    text_weights=TextWeights(
        weights={
            "name": 3.0,        # 상품명 3배 가중
            "brand": 2.0,       # 브랜드 2배 가중
            "description": 1.0  # 설명 기본 가중
        }
    )
)

# Scoring Profile이 이미 있는지 확인
profile_exists = False
if existing_index.scoring_profiles:
    profile_exists = any(p.name == "product-weighted-profile" 
                        for p in existing_index.scoring_profiles)

if profile_exists:
    print("⚠️ 기존 Scoring Profile이 이미 존재합니다.")
else:
    # Scoring Profile 추가
    if existing_index.scoring_profiles is None:
        existing_index.scoring_profiles = []
    
    existing_index.scoring_profiles.append(scoring_profile)
    
    # 인덱스 업데이트
    index_client.create_or_update_index(existing_index)
    print("✅ Scoring Profile 'product-weighted-profile' 생성 완료")
    print()
    print("가중치 설정:")
    print("  - name: 3.0 (상품명)")
    print("  - brand: 2.0 (브랜드)")
    print("  - description: 1.0 (설명)")

# 최종 확인
updated_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)
print(f"\n현재 인덱스의 Scoring Profiles: {[p.name for p in updated_index.scoring_profiles] if updated_index.scoring_profiles else '없음'}")

✅ Scoring Profile 'product-weighted-profile' 생성 완료

가중치 설정:
  - name: 3.0 (상품명)
  - brand: 2.0 (브랜드)
  - description: 1.0 (설명)

현재 인덱스의 Scoring Profiles: ['product-weighted-profile']


## 4. Scoring Profile 적용 검색

생성한 Scoring Profile을 적용하여 검색합니다.

In [5]:
##############################################
# 4. Scoring Profile 적용 검색
##############################################
search_query = "튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함."

results = search_client.search(
    search_text=search_query,
    top=3,
    search_fields=["name", "brand", "description"],
    scoring_profile="product-weighted-profile",  # Scoring Profile 적용
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[Scoring Profile 적용 검색 결과]")
print("프로필: product-weighted-profile")
print("가중치: name(3.0), brand(2.0), description(1.0)")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   검색 점수: {score}")
    print(f"   가격: ₩{result['price']:,}")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

검색어: '튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함.'

[Scoring Profile 적용 검색 결과]
프로필: product-weighted-profile
가중치: name(3.0), brand(2.0), description(1.0)

1. 캔버스 백팩 (어반트래블)
   검색 점수: 5.159078
   가격: ₩79,000.0
   설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.


## 5. Scoring Profile 효과 비교

동일한 검색어로 Scoring Profile 적용 전후를 비교하여 순위 변화를 확인합니다.

In [6]:
##############################################
# 6. Scoring Profile 효과 비교
##############################################
search_query = "튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함."

# 6-1. Scoring Profile 없음
results_no_profile = list(search_client.search(
    search_text=search_query,
    top=3,
    search_fields=["name", "brand", "description"],
    select=["id", "name", "brand", "price", "description"]
))

# 6-2. Scoring Profile 적용
results_with_profile = list(search_client.search(
    search_text=search_query,
    top=3,
    search_fields=["name", "brand", "description"],
    scoring_profile="product-weighted-profile",
    select=["id", "name", "brand", "price", "description"]
))

print(f"검색어: '{search_query}'")
print(f"\n{'='*80}")
print("[Scoring Profile 효과 비교]")
print(f"{'='*80}\n")

print("📊 Scoring Profile 없음 (기본)")
print("-" * 80)
for idx, result in enumerate(results_no_profile, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. [{score:>6}] {result['name']:<40} | {result['brand']:<15}")
    print(f"{result['description']:}")

print(f"\n📈 Scoring Profile 적용 (name: 3.0, brand: 2.0, description: 1.0)")
print("-" * 80)
for idx, result in enumerate(results_with_profile, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. [{score:>6}] {result['name']:<40} | {result['brand']:<15}")
    print(f"{result['description']:}")

print(f"\n{'='*80}")
print("💡 Scoring Profile을 적용하면 상품명과 브랜드에서 매칭된 항목이 상위에 노출됩니다.")

검색어: '튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함.'

[Scoring Profile 효과 비교]

📊 Scoring Profile 없음 (기본)
--------------------------------------------------------------------------------
1. [3.2723968] 캔버스 백팩                                   | 어반트래블          
내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.

📈 Scoring Profile 적용 (name: 3.0, brand: 2.0, description: 1.0)
--------------------------------------------------------------------------------
1. [5.159078] 캔버스 백팩                                   | 어반트래블          
내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.

💡 Scoring Profile을 적용하면 상품명과 브랜드에서 매칭된 항목이 상위에 노출됩니다.


## 6. 하이브리드 검색 + Scoring Profile

키워드 검색에 Scoring Profile을 적용하고, 벡터 검색을 결합하여 최상의 검색 결과를 얻습니다.

In [7]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 7. 하이브리드 검색 + Scoring Profile
##############################################
search_query = "튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함."

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=5,
    search_fields=["name", "brand", "description"],
    scoring_profile="product-weighted-profile",  # Scoring Profile 적용
    select=["id", "name", "brand", "price", "description", "imageUrl"]
)

print(f"검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[하이브리드 검색 + Scoring Profile]")
print("키워드 검색: product-weighted-profile")
print("벡터 검색: descriptionVector (의미 기반 검색)")
print(f"{'='*60}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    print(f"{idx}. {result['name']} ({result['brand']})")
    print(f"   통합 점수: {score}")
    print(f"   가격: ₩{result['price']:,}")
    print(f"   설명: {result['description']}")
    display(HTML(f'<img src="{result["imageUrl"]}" width="200" />'))
    print()

검색어: '튼튼하게 만들어진 백팩으로, 노트북을 넣을 수 있어야함.'

[하이브리드 검색 + Scoring Profile]
키워드 검색: product-weighted-profile
벡터 검색: descriptionVector (의미 기반 검색)

1. 캔버스 백팩 (어반트래블)
   통합 점수: 0.03333333507180214
   가격: ₩79,000.0
   설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.



2. 숄더백 (어바에센셜)
   통합 점수: 0.016393441706895828
   가격: ₩159,000.0
   설명: 심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다.



3. 가죽 크로스백 (럭셔리레더)
   통합 점수: 0.016129031777381897
   가격: ₩189,000.0
   설명: 고급 천연 가죽으로 제작된 크로스백입니다. 심플한 디자인으로 어떤 스타일과도 잘 어울립니다.



4. 패딩 점퍼 (윈터프루프)
   통합 점수: 0.01587301678955555
   가격: ₩245,000.0
   설명: 구스다운 충전재를 사용한 경량 패딩 점퍼입니다. 보온성이 뛰어나면서도 가볍습니다.



5. 코튼 조거 팬츠 (컴포트웨어)
   통합 점수: 0.015625
   가격: ₩49,000.0
   설명: 부드러운 코튼 소재의 조거 팬츠입니다. 홈웨어로도, 가볍운 외출복으로도 활용 가능합니다.


## 7. Scoring Profile 제거 (선택사항)

In [8]:
##############################################
# 7. Scoring Profile 제거 (테스트용)
##############################################

DELETE_SCORING_PROFILE = False

if DELETE_SCORING_PROFILE:
    # 기존 인덱스 가져오기
    existing_index = index_client.get_index(AZURE_SEARCH_INDEX_NAME)

    # product-weighted-profile 제거
    if existing_index.scoring_profiles:
        print("\n⚠️ 'product-weighted-profile' 제거 중...")
        
        # 해당 profile 제거
        original_count = len(existing_index.scoring_profiles)
        existing_index.scoring_profiles = [
            profile for profile in existing_index.scoring_profiles 
            if profile.name != "product-weighted-profile"
        ]
        removed_count = original_count - len(existing_index.scoring_profiles)
        
        try:
            # 인덱스 업데이트
            index_client.create_or_update_index(existing_index)
            print(f"✅ {removed_count}개의 Scoring Profile 제거 완료")
        except Exception as e:
            print(f"❌ 제거 실패: {e}")
            raise
    else:
        print("\nℹ️  제거할 Scoring Profile이 없습니다.")